# Ejercicio de Mkt Movement

En este ejercicio se quiere demostrar el problema del mkt ovement usando curvas con futuros y usanod curvas solo con swaps. 

Para empezar, se importará las curvas anteriores del archivo ``curvas.py``. Tambien se importarán las librerías necesarias para este ejercicio:
- pandas
- numpy
- QuantLib 

In [1]:
# importacion de librerías
import pandas as pd
import numpy as np
import QuantLib as ql

# Importamos el módulo de curvas
import curvas as curvs


A continuación se definirá la función para crear swaps por medio de QuantLib:

In [2]:
def ftiieSwap(start, maturity, notional, rate, spread,  typ, rule, float_index, discount_engine):
    """
    Crea un swap de TIIE de Fondeo o TIIE28 con los parámetros dados.

    Parameters
    ----------
    start : ql.Date
        Fecha de inicio del swap.
    maturity : ql.Date
        Fecha de vencimiento del swap.
    notional : float
        Monto nocional del swap.
    rate : float
        Tasa fija del swap.
    spread : float
        Spread a aplicar al swap.
    typ : ql.Swap.Type
        Tipo de swap (Fixed vs Floating).
    rule : ql.DateGeneration.Rule
        Regla de generación de fechas para el swap.
    float_index : ql.OvernightIndex
        Índice de referencia para la parte flotante del swap.
    discount_engine : ql.PricingEngine
        Motor de valoración para el swap.

    Returns
    -------
    ql.OvernightIndexedSwap
        Objeto swap configurado con los parámetros dados.
    """
    # Parameters Definition
    cal = ql.Mexico() # Calendario
    legDC = ql.Actual360() # Convención de conteo de días
    cpn_tenor = ql.Period(13) # Periodo de pagos se divide el año en 13
    convention = ql.Following # Convención para días inhábiles (Following)
    termDateConvention = ql.Following
    isEndOfMonth = False # Si debe de terminar siempre en fin de mes
    pay_days = 2 # Dias en los que se paga despues del corte

    # Se crea el calendario de CUPONES
    fixfltSchdl = ql.Schedule(start, maturity, cpn_tenor, cal, convention, termDateConvention, rule, isEndOfMonth)
    # Se crea el Swap
    swap = ql.OvernightIndexedSwap(typ, notional, fixfltSchdl, 
                                   rate, legDC, float_index, 
                                   spread, pay_days,  
                                   convention, cal)
    # le asignamos la curva de descuento
    swap.setPricingEngine(discount_engine)

    return swap

Se creará una función para añadir nodos a la curva de Swaps ya obtenida. 
Estos nodos son las tasas par que salen de la curva FTIIE creada con futuros.

In [3]:

def aniade_tenors(dt_today, tenors_a_aniadir, indice_variable, motor_descuento):
    """
    Añade tenores a una lista de tenores existentes.

    Parameters
    ----------
    tenors_a_aniadir : list
        Lista de tenores a añadir (ej. ['1M', '3M', '6M']).
    indice_variable : ql.OvernightIndex
        Índice de referencia para el swap (ej. ql.Mexico().TIIE_28).
    motor_descuento : ql.PricingEngine
        Motor de valoración para el swap (ej. ql.DiscountingSwapEngine).

    Returns
    -------
    list
        Lista actualizada con los nuevos tenores.
    """

    qleval_date = ql.Date().from_date(dt_today)
    
    # Como es para la curva FTIIE, usamos 2 días de spot
    spot_date = ql.Mexico().advance(qleval_date, ql.Period('2D')) # dia que empiezan los swaps (t+2)
    

    fair_rates = []
    for ten in tenors_a_aniadir:

        maturity = spot_date + 28*int(ten[:-1])

        notional = 10_000_000
        rate = 9.5/100
        typ = -1 # Recibido 
        spread = 0
        rule = 0 # Backwards

        swap = ftiieSwap(spot_date, maturity, 
                         notional, rate, spread, 
                         typ, rule, indice_variable, 
                         motor_descuento)

        fair_rate = swap.fairRate()*100 # calcular la tasa par

        fair_rates.append(fair_rate) # Añadir la tasa par a la lista
        

    return fair_rates    

Se calculan las curvas al cierre de 19 de febrero de 2025
Comenzamos por la curva de SOFR

In [4]:
# Empezamos por la curva de SOFR

# Se define la fecha de evaluación
dt_today = pd.to_datetime('2025-02-19')
qleval_date = ql.Date().from_date(dt_today)
ql.Settings.instance().evaluationDate = qleval_date

# Ahora definimos los inputs que necesitamos para crear la curva de SOFR 
tasas_SOFR = curvs.sofr_swaps
tenors_SOFR = curvs.tenors_sofr
fut_SOFR = curvs.sofr_futures
tenors_fut_SOFR = curvs.tenors_fut_sofr
zero_SOFR_tenor = curvs.tenor_depo
zero_SOFR = curvs.depo 

# Generamos la curva de SOFR
crvSOFR = curvs.genSOFR(fut_SOFR, tenors_fut_SOFR, tasas_SOFR, tenors_SOFR, zero_SOFR, zero_SOFR_tenor)




Seguimos con la curva de Descuento mexicana o DISCMXN

Igual se define primero las tasas que se usaran y luego genraremos la curva

In [18]:
# Se definen los insumos para la curva de descuento mexicana
tasas_ftiie = curvs.tasas_ftiie
tenors_ftiie = curvs.tenors_ftiie
tasas_xccy = curvs.tasas_xccy
tenors_xccy = curvs.tenors_xccy
tasas_fut = curvs.tasas_fwd_fx
tenors_fut = curvs.tenors_fwd_fx
spot_fx = curvs.t_mxn_usd_spot

# Ahora que tenemos las nuevas tasas y tenors, las agregamos a `tasas_ftiie` y `tenors_ftiie` para crear una curva solo con swaps
tenor_tasa_ftiie_dict = dict(zip(tenors_ftiie, tasas_ftiie))

# Obtén las tasas para los tenors_fut
tasas_ftiie_para_fut = [tenor_tasa_ftiie_dict.get(t, None) for t in tenors_fut]
tasas_ftiie_para_fut

# Ahora creamos la curva de descuento mexicana
crvDISCMXN = curvs.genDISCTIIE(spot_fx, tasas_fut, tenors_fut,
                                tasas_xccy, tenors_xccy,
                                tasas_ftiie,
                                crvSOFR)

Por último se define la curva de FTIIE, priemro se definen sus cierres y luegi se construye la curva 

In [19]:
tenors_fut_ftiie = curvs.tenors_futuros
tasas_fut_ftiie = curvs.tasas_fut
fechas_banxico = curvs.fechas_banxico
tasas_banxico = curvs.tasas_banxico

# Ahora creamos la curva de FTIIE
crvFTIIE = curvs.genFTIIE(tasas_ftiie, tenors_ftiie, crvDISCMXN,
                          tasas_fut_ftiie, tenors_fut_ftiie,
                          fechas_banxico, tasas_banxico)


Ahora calculamos las tasas de los tenors 1, 2 y 156, es decir 1 mes, 2 meses y 12 años. Se toman estos tenors porque son tenors operables en el mercado de swaps.

Nuestra meta es calcular el riesgo de un swap, pero para esto, necesitamos todos los nodos operables de la curva.

In [7]:
tenors_a_aniadir = ['1M', '2M', '156M']  # Tenors a añadir
dias_liq = 2  # Días de liquidación
moneda = ql.MXNCurrency()
calendario = ql.Mexico()  # Calendario de México
cont_dias = ql.Actual360()  # Convención de conteo de días

estructura_curva_swap = ql.YieldTermStructureHandle(crvFTIIE)

index_MXNFTIIE = ql.OvernightIndex(
        'SWP_FTIIE',
        dias_liq, 
        moneda, calendario,  
        cont_dias, 
        estructura_curva_swap)

# motor de descuento para los swaps
motor_descuento = ql.DiscountingSwapEngine(ql.YieldTermStructureHandle(crvDISCMXN))

# Añadimos los tenores a la curva de FTIIE
tasas_fair = aniade_tenors(dt_today, tenors_a_aniadir, index_MXNFTIIE, motor_descuento)
tasas_fair

[9.544013429312296, 9.405434560379994, 8.944868729129562]

Ahora que tenemos las nuevas tasas y tenors, las agregamos a `tasas_ftiie` y `tenors_ftiie` para crear una curva solo con swaps


In [8]:
tenors_ftiie

[3, 6, 9, 13, 26, 39, 52, 65, 91, 130, 195, 260, 390]

In [15]:
# Ahora vamos a añadir los tenores de FTIIE que no tenemos
tenors_a_aniadir_int = [int(t.replace('M', '')) for t in tenors_a_aniadir]
nuevos_tenors_ftiie = tenors_ftiie + tenors_a_aniadir_int

tenors_ftiie_ordenados = sorted(list(set(nuevos_tenors_ftiie)))

# Combina los tenors y tasas originales con los nuevos
nuevas_tasas_ftiie = tasas_ftiie + tasas_fair
nuevos_tenors_ftiie = tenors_ftiie + [int(t.replace('M', '')) for t in tenors_a_aniadir]

# Crea un diccionario para emparejar tenor con tasa
tenor_tasa_dict = dict(zip(nuevos_tenors_ftiie, nuevas_tasas_ftiie))

# Ordena las tasas según tenors_ftiie_ordenados
tasas_ftiie_ordenadas = [tenor_tasa_dict[t] for t in tenors_ftiie_ordenados]
list(zip(tenors_ftiie_ordenados,tasas_ftiie_ordenadas))


[(1, 9.544013429312296),
 (2, 9.405434560379994),
 (3, 9.27),
 (6, 8.943),
 (9, 8.74025),
 (13, 8.5945),
 (26, 8.45),
 (39, 8.4568),
 (52, 8.5125),
 (65, 8.5655),
 (91, 8.6935),
 (130, 8.85225),
 (156, 8.944868729129562),
 (195, 9.03405),
 (260, 9.051),
 (390, 8.98505)]

A continuación se crea de nuevo la curva de FTIIE pero solo usando las tasas y tenors nuevos de ftiie

In [21]:
crvFTIIE_nueva = curvs.genFTIIE(
    tasas_ftiie_ordenadas, tenors_ftiie_ordenados, crvDISCMXN)

pd.DataFrame(crvFTIIE_nueva.nodes()).rename(columns={0: 'Fecha', 1: 'Factor de Descuento'})  # DataFrame con los nodos de la curva de descuento de FTIIE

,Fecha,Factor de Descuento
0,"February 19th, 2025",1.000000
1,"March 25th, 2025",0.991069
2,"April 23rd, 2025",0.983745
3,"May 20th, 2025",0.977207
4,"August 12th, 2025",0.957882
5,"November 4th, 2025",0.939536
6,"February 24th, 2026",0.915799
7,"February 23rd, 2027",0.842361
8,"February 22nd, 2028",0.773388
9,"February 20th, 2029",0.708374


Ahora se definirá una función que nos calculará una curva para cada Tenor sumando y restando un punto base. A esta curva le llamaremos `genKRRcurves`


In [ ]:
def genKRRcurves(tasas_ftiie_ordenadas, tenors_ftiie_ordenados, crvSOFR,
                 tasas_fwd_fx=None, tenors_fwd_fx=None, tasas_xccy=None, tenors_xccy=None, puntos_base=1):
    """
    Genera curvas KRR para cada tenor sumando y restando puntos base.

    Parameters
    ----------
    crvFTIIE : Curva de descuento de FTIIE.
    tenors_ftiie_ordenados : list
        Lista de tenores ordenados.
    puntos_base : int, optional
        Número de puntos base a sumar/restar. Por defecto es 1.

    Returns
    -------
    dict
        Diccionario con las curvas KRR para cada tenor.
    """
    curvas_krr = {}
    
    for i in range(tenors_ftiie_ordenados):
        
        tasa_mas = tasas_ftiie_ordenadas[i] + 0.01  # Sumar 1 punto base
        tasa_menos = tasas_ftiie_ordenados[i] - 0.01  # Restar 1 punto base

        nuevas_tasas_mas = tasas_ftiie_ordenadas[:i] + [tasa_mas] + tasas_ftiie_ordenadas[i+1:]
        nuevas_tasas_menos = tasas_ftiie_ordenadas[:i] + [tasa_menos] + tasas_ftiie_ordenadas[i+1:]
        

        


    
    return curvas_krr